In [62]:
import numpy as np
import time
import cv2
from pathInertiaModel import pathInertiaModel
from socket import socket, gethostbyname, AF_INET, SOCK_DGRAM
from sklearn.neighbors import KNeighborsClassifier
import copy
from random import randint
import skimage.measure
import keras
import math
import threading
import random
from skimage import io
from sklearn.cluster import KMeans
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon


kernel = np.ones((5,5),np.uint8)
blnk = cv2.imread('blank.png', 0)
blnk = cv2.resize(blnk,(260,680))
centroidsAll = [[0,0,0,0]]
scr = None
grayG = None
hsv=None

class camshift:
#     r,h,c,w = 0,10,360,10
    ret = [] 
    hsv_roi = None
    track_window = None
    mask = None
    roi_hist = None
    term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1 )
    coord = []
    trackcount = 0
    cnt = 10
    has = True
    global hsv, loadModel, scr, grayG
    def  __init__(self,cx, cy):
        try:
            x, y, h, w = (cx-20), (cy-20), 40, 40 
            self.roi = hsv[y:y+w, x:x+h,:]
#             mean = np.mean(self.roi)
# #             self.hsv_roi =  cv2.cvtColor(self.roi, cv2.COLOR_BGR2GRAY)
#             mask = cv2.inRange(self.roi,np.array([mean-5]), np.array([mean+5]))
            self.roi_hist = cv2.calcHist([self.roi], [0,1], None, [180,255], [0,180,0,255])
            cv2.normalize(self.roi_hist,self.roi_hist,0,255,cv2.NORM_MINMAX)
            self.track_window = (x,y,w,h)
            self.cnt=random.randint(10,12)
            self.has = True
        except:
            self.has = False
    def comp(self):
#         pts[1][1]-pts[0][1]
#         pts[1][0]-pts[0][0]
#         pts[2][1]-pts[1][1]
#         pts[2][0]-pts[1][0]
        if self.trackcount<=self.cnt:
            return False#remain
        else:
            return True#delete
    def track_csft(self):
        try:       
            dst = cv2.calcBackProject([hsv],[0,1],self.roi_hist,[0,180,0,255],1)
#             print(ret)
            self.ret, self.track_window = cv2.meanShift(dst, self.track_window, self.term_crit)
          
            self.trackcount += 1
        except:
            self.trackcount += 1
#         print(self.ret)
    def preTrackPred_csft(self):
        loadModel = None
    def coord(self):
        x,y,w,h =  self.track_window
        return [[x,y],[x+w,y],[x+w,y+h],[x,y+h]]
#         img2 = cv2.polylines(frame,[pts],True, 255,2)
#         cv2.imshow('img2',img2)        
        
    
def sift_knn(img1,img2,part1,part2):
    
    global centroidsAll
    infolist = []
    sift = cv2.xfeatures2d.SURF_create()
    kp1 = sift.detect(img1,None)
    kp1, des1 = sift.compute(img1, kp1)
    kp2 = sift.detect(img2,None)
    kp2, des2 = sift.compute(img2, kp2)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    matchesMask = [[0,0] for i in range(len(matches))]
          
    for i, (m1,m2) in enumerate(matches):
        if m1.distance < 0.9 * m2.distance:
            matchesMask[i] = [1,0]
            pt1 = kp1[m1.queryIdx].pt
            pt2 = kp2[m1.trainIdx].pt
#             col = list(img2[int(pt2[1])-1 : int(pt2[1])+1,int(pt2[0])-1 : int(pt2[0])+1,:].flatten())
            info = [10*pt2[0], 10*pt2[1], 50*(pt2[1]-pt1[1]), 50*(pt2[0]-pt1[0])]#+col
#             print("yeah:",img2[int(pt2[0])-1 : int(pt2[0])+1,int(pt2[1])-1 : int(pt2[1])+1,:].flatten())
            infolist.append(info)
            cv2.circle(img1, (int(pt1[0]),int(pt1[1])), 1, (255,0,255), -1)
            cv2.circle(img2, (int(pt2[0]),int(pt2[1])), 1, (0,255,255), -1)



    draw_params = dict(matchColor = (255, 0,0),
                       singlePointColor = (0,0,255),
                       matchesMask = matchesMask,
                       flags = 2)
    res = cv2.drawMatchesKnn(img2,kp1,img2,kp2,matches,None,**draw_params)    
    
    infolist = np.array(infolist).reshape(-1,4)
    nc = min(infolist.shape[0],10)
    
    kmeans = KMeans(n_clusters = nc, n_init=10, max_iter=100)
    kmeans.fit(infolist)
    centroid = kmeans.cluster_centers_
        
    for inx in range(len(centroid)):
        cv2.circle(img2,(int(centroid[inx][0]/10),int(centroid[inx][1]/10)),5,(0,0,255),-1)
#         centroid[inx][0]+=part1
#         centroid[inx][1]+=part2
    centroidsAll += list(np.array(centroid)+[10*part2,10*part1,0,0])
    
    
        
def obstacleNcounters():
    PORT_NUMBER = 4000
    SIZE = 1024*1024
#    hostName = gethostbyname( '0.0.0.0' )
#    mySocket = socket( AF_INET, SOCK_DGRAM )
#    mySocket.bind( (hostName, PORT_NUMBER) )
    print ("Test server listening on port {0}\n".format(PORT_NUMBER))
    
    print("start")
    global centroidsAll, grayG
    vrec = cv2.VideoCapture('v4.mp4',0)
    k = 300*8
    prev1 = []
    shape = [260,680]
    cfstObj = []
    global loaded_model, hsv, scr
    
    with open("pathInertia.json", "r") as json_file:
        loaded_model_json = json_file.read()
        loaded_model = keras.models.model_from_json(loaded_model_json)
        loaded_model.load_weights("pathInertia.h5")
        print("Model loaded from disk")
###
    while k:
        ret, scr = vrec.read()
        k-=1
    print("running")
##    
    while True:
        
        start_time = time.time()
        centroidsAll = []
        ret, scr = vrec.read()
        scr1 = scr
        if not ret:
            break
    
        scr = cv2.resize(scr, (680,480)) 
        scr = np.array(scr[220:480,:])
        scr = cv2.resize(scr, (shape[1],shape[0]))#1360,520
        hsv = cv2.cvtColor(scr, cv2.COLOR_BGR2HSV)
        grayG = cv2.cvtColor(scr, cv2.COLOR_BGR2GRAY)
        blank = cv2.imread('blank.png', 0)
        blank = cv2.resize(blank,(shape[1],shape[0]))
        cv2.imshow('hsv0',grayG)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(gray,50,100)
        canny = blank - canny
        
        scr1 = canny[0:shape[0]//2, 0:shape[1]//3]
        scr2 = canny[0:shape[0]//2,shape[1]//3:2*shape[1]//3]
        scr3 = canny[0:shape[0]//2,2*shape[1]//3:shape[1]]
        scr4 = canny[shape[0]//2:shape[0], 0:shape[1]//3]
        scr5 = canny[shape[0]//2:shape[0],shape[1]//3:2*shape[1]//3]
        scr6 = canny[shape[0]//2:shape[0],2*shape[1]//3:shape[1]]
        if prev1 == [] :
            prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1 = threading.Thread(target = sift_knn, args=(prev1,scr1,0,0))#,0,0))
        th2 = threading.Thread(target = sift_knn, args=(prev2,scr2,0,shape[1]//3))#,0,shape[1]//3))
        th3 = threading.Thread(target = sift_knn, args=(prev3,scr3,0,2*shape[1]//3))#,0,2*shape[1]//3))
        th4 = threading.Thread(target = sift_knn, args=(prev4,scr4,shape[0]//2,0))#,shape[0]//2,0))
        th5 = threading.Thread(target = sift_knn, args=(prev5,scr5,shape[0]//2,shape[1]//3))#,shape[0]//2,shape[1]//3))
        th6 = threading.Thread(target = sift_knn, args=(prev6,scr6,shape[0]//2,2*shape[1]//3))#,shape[0]//2,2*shape[1]//3))
        th1.start()
        th2.start()
        th3.start()
        th4.start()
        th5.start()
        th6.start()
        
        frame = cv2.GaussianBlur(scr, (3, 3), 0)
        scr = cv2.addWeighted(scr,10,frame,-9,0)
        cv2.imshow('scred',scr)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        Cy = cv2.Canny(gray,50,100)
        Cy = blank - Cy
        im2, contours, hierarchy = cv2.findContours(Cy, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        blank = cv2.drawContours(blank, contours, -1, 0, 1)
        dist_transform = cv2.distanceTransform(Cy,cv2.DIST_L2,5)
        ret, blank = cv2.threshold(dist_transform,0.02*dist_transform.max(),255,0)
        blank = cv2.morphologyEx(blank, cv2.MORPH_OPEN, kernel,3)
        cv2.imshow("blk1",blank)
        blank = np.uint8(blank)
        im3, contours1, hierarchy = cv2.findContours(blank, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        scr = cv2.drawContours(scr, contours1, -1, 0, 1)
        cv2.imshow("blk2",scr)
#         print("no",len(cfstObj))
        for obj in cfstObj:
            obj.track_csft()
            obj.preTrackPred_csft()
            x,y,w,h = obj.track_window
            cv2.rectangle(scr, (x,y), (x+w,y+h), [0,0,255],2)
            if obj.comp():
                cfstObj.remove(obj)
        
        prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1.join()
        th2.join()
        th3.join()
        th4.join()
        th5.join()
        th6.join()
        cv2.imshow('print',canny)
        for cnt in centroidsAll:
            point = Point(cnt[0]/10,cnt[1]/10)
            has=1
            for obj in cfstObj:
#                 pts = cv2.boxPoints(obj.ret)
                polygon = Polygon(obj.coord())
                if polygon.contains(point):
                    has=0
                    break
            if has:    
                tempobj = camshift(int(cnt[0]/10),int(cnt[1]/10))
                retCt = tempobj.has
                if retCt:  
                    tempobj.track_csft()
                    x,y,w,h = tempobj.track_window        
                    cv2.rectangle(scr, (x,y), (x+w,y+h), [225,0,255],2)
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(scr,str(x)+str(y),(x,y), font, 1,(255,255,0),1,cv2.LINE_AA)

#                     tempobj.preTrackPred_csft()
                    cfstObj.append(tempobj)
            
                    
                
#         kmeanC = KMeans(n_clusters = 10, n_init=10, max_iter=100)
#         kmeanC.fit(centroidsAll)
#         centroid = kmeanC.cluster_centers_
#         for cnt in centroid:
#              cv2.circle(canny,(int(cnt[0]/2),int(cnt[1]/2)),10,(0,0,255),-1)
        cv2.imshow('print2',canny) 
        cv2.imshow('print3',scr)   
 

        print("FPS: ", 1.0 / (time.time() - start_time))
        q=cv2.waitKey(1)
        if q == 27:
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
obstacleNcounters()

Test server listening on port 4000

start
Model loaded from disk
running
FPS:  1.3432970524228998


/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:190: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


FPS:  2.33565768444734
FPS:  2.15833985574031
FPS:  2.1280577831851986
FPS:  2.1365641302282556
FPS:  2.0893181017773856
FPS:  2.032730698696898
FPS:  1.929641730975728
FPS:  1.854998624552316
FPS:  1.991132176148782
FPS:  1.832179458449802
FPS:  1.9018034990829473
FPS:  1.942052861310712
FPS:  1.9503138457131777
FPS:  1.9525754997001985
FPS:  1.9891397238549522
FPS:  1.8790287102303322
FPS:  2.052078104436062
FPS:  1.9531892560955904
FPS:  2.044007797270955
FPS:  2.024405983196814
FPS:  2.0684022430231006
FPS:  1.9197770766669702
FPS:  2.0253326901412465
FPS:  1.7864537502161564
FPS:  1.7648196871269648
FPS:  1.8287452333552208
FPS:  1.6422329460361587
FPS:  1.815649207713136
FPS:  1.7946456345221322
FPS:  1.6573527348882238
FPS:  1.892907302103078
FPS:  1.903848545617789
FPS:  1.7864453804311946
FPS:  1.8292604636232956
FPS:  1.8177588320034117
FPS:  1.921369726465156


In [4]:
ret = ((22.0, 24.0), (44.10992431640625, 47.42926788330078), 164.38218688964844)
pts = cv2.boxPoints(ret)
pts = np.int0(pts)
# cnt = np.array([[0,0], [1,1], [1,0],[0,1]])
# bbox = cv2.minAreaRect(cnt)
# pts = cv2.boxPoints(bbox)
print(pts)

[[36 -4]
 [49 40]
 [ 7 52]
 [-5  7]]


In [25]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

point = Point(0.5, 0.5)
polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
print(polygon.contains(point))

True


In [46]:
a=list(np.array([1,2])+[7,8])
print(a)

[8, 10]


In [ ]:
def sift_knn(img1,img2):
    global infoAll
    infolist = []
    sift = cv2.xfeatures2d.SURF_create()
    kp1 = sift.detect(img1,None)
    kp1, des1 = sift.compute(img1, kp1)
    kp2 = sift.detect(img2,None)
    kp2, des2 = sift.compute(img2, kp2)
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    matchesMask = [[0,0] for i in range(len(matches))]
          
    # ratio test as per Lowe's paper
    j=0
    k=0    
    for i, (m1,m2) in enumerate(matches):
        k+=1
        if m1.distance < 0.7 * m2.distance:
            j+=1
            matchesMask[i] = [1,0]
            pt1 = kp1[m1.queryIdx].pt
            pt2 = kp2[m1.trainIdx].pt
            info = [lambda pt1, pt2: [10*pt2[0], 10*pt2[1], 10*(pt2[1]-pt1[1]), 10*(pt2[0]-pt1[0]), img2[pt2[0]-5 : pt2[0]+5,pt2[1]-5 : pt2[1]+5].flatten()]]
            infolist.append(info)
#             print(np.array(info).shape)
            cv2.circle(img1, (int(pt1[0]),int(pt1[1])), 1, (255,0,255), -1)
            cv2.circle(img2, (int(pt2[0]),int(pt2[1])), 1, (0,255,255), -1)
        
#     print(j)
#     print(k)
    # Draw match in blue, error in red
    draw_params = dict(matchColor = (255, 0,0),
                       singlePointColor = (0,0,255),
                       matchesMask = matchesMask,
                       flags = 0)
    res = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
#     cv2.imshow("res",res)
    try:
        infoAll.append(infolist)
    except:
        time.sleep(0.001*random.randint(1,9))
        infoAll.append(infolist)
        

In [14]:
scr = cv2.imread('c1.png',1)
cv2.imshow("src",scr)
r,h,c,w = (500-15), 30, (500-15), 30 
roi = scr[r:r+h, c:c+w]
cv2.imshow("roi",roi)
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
cv2.imshow("hsv_roi",hsv_roi)
mask = cv2.inRange(hsv_roi,np.array([0, 30, 30]), np.array([180,255,255]))
cv2.imshow("mask",mask)
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
# cv2.imshow("hsv_roi",hsv_roi)
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
track_window = (c,r,w,h)
q=cv2.waitKey(1)

In [ ]:

import numpy as np
import cv2
import sys
video_path = 'v4.mp4'
cv2.ocl.setUseOpenCL(False)
version = cv2.__version__.split('.')[0]
print(version)
#read video file
cap = cv2.VideoCapture(video_path)
#check opencv version
if version == '2' :
    fgbg = cv2.BackgroundSubtractorMOG2()
if version == '3':
    fgbg = cv2.createBackgroundSubtractorMOG2()
while (cap.isOpened):
    #if ret is true than no error with cap.isOpened
    ret, frame = cap.read()
    if ret==True:
    #apply background substraction
        fgmask = fgbg.apply(frame)
    #check opencv version
    if version == '2' :
        (contours, hierarchy) = cv2.findContours(fgmask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if version == '3' :
        (im2, contours, hierarchy) = cv2.findContours(fgmask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    #looping for contours
    for c in contours:
        if cv2.contourArea(c) < 500:
            continue
    #get bounding box from countour
        (x, y, w, h) = cv2.boundingRect(c)
    #draw bounding box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.imshow('foreground and background',fgmask)
        cv2.imshow('rgb',frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

3


In [11]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

# take first frame of the video
ret,frame = cap.read()

# setup initial location of window
r,h,c,w = 250,90,400,125  # simply hardcoded the values
track_window = (c, r, w, h)

# set up the ROI for tracking
roi = frame[r:r+h, c:c+w]
hsv_roi =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 0.,0.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while(1):
    ret ,frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        print(ret)
        # Draw it on image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 0,5)
        cv2.imshow('img2',img2)
        k = cv2.waitKey(1)
        if k == 27:
            break

cv2.destroyAllWindows()
cap.release()

10
10
7
6
5
8
9
7
0
9
7
4
3
10
9
10
7
6
10
3
10
3
5
10
4
6
7
4
10
9
7
7
6
8
10
10
2
4
6
10
10
10
10


In [22]:
import numpy as np
import cv2

cap = cv2.VideoCapture("v4.mp4",0)

# take first frame of the video
ret,frame = cap.read()

# setup initial location of window
# r,h,c,w = 100,40,100,40  # simply hardcoded the values
# track_window = (c,r,w,h)


# # set up the ROI for tracking
# roi = frame[r:r+h, c:c+w]
# hsv_roi =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
# # mask = cv2.inRange(hsv_roi, np.array((0., 0.,0.)), np.array((180.,255.,255.)))
# roi_hist = cv2.calcHist([hsv_roi],[0],None,[256],[0,255])
# cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# # Setup the termination criteria, either 10 iteration or move by atleast 1 pt
# term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while(1):
    ret ,frame = cap.read()

    if ret == True:
#         hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,255],1)
#         # apply meanshift to get the new location
#         ret, track_window = cv2.meanShift(dst, track_window, term_crit)

#         # Draw it on image
#         x,y,w,h = track_window
#         img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 0,3)
# #         # apply meanshift to get the new location
# #         ret, track_window = cv2.CamShift(dst, track_window, term_crit)

# #         # Draw it on image
# #         pts = cv2.boxPoints(ret)
# #         pts = np.int0(pts)
# #         img2 = cv2.polylines(frame,[pts],True, 255,2)
        cv2.line(frame, (0,100),(0,200),[0,0,0],5)
        cv2.imshow('img2',frame)
        k = cv2.waitKey(1)
        if k == 27:
            break

cv2.destroyAllWindows()
cap.release()

10


In [9]:
import numpy as np
import time
import cv2
from pathInertiaModel import pathInertiaModel
from socket import socket, gethostbyname, AF_INET, SOCK_DGRAM
from sklearn.neighbors import KNeighborsClassifier
import copy
from random import randint
import skimage.measure
import keras
import math
import threading
import random
from skimage import io
from sklearn.cluster import KMeans
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon


kernel = np.ones((5,5),np.uint8)
blnk = cv2.imread('blank.png', 0)
blnk = cv2.resize(blnk,(260,680))
centroidsAll = [[0,0,0,0]]
scr = None
grayG = None
hsv=None

class camshift:
#     r,h,c,w = 0,10,360,10
    ret = [] 
    hsv_roi = None
    track_window = None
    mask = None
    roi_hist = None
    term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1 )
    coord = []
    trackcount = 0
    cnt = 10
    has = True
    global hsv, loadModel, scr, grayG
    def  __init__(self,cx, cy):
        try:
            r, h, c, w = (cx-20), 40, (cy-20), 40 
            self.roi = hsv[r:r+h, c:c+w,:]
#             mean = np.mean(self.roi)
# #             self.hsv_roi =  cv2.cvtColor(self.roi, cv2.COLOR_BGR2GRAY)
#             mask = cv2.inRange(self.roi,np.array([mean-5]), np.array([mean+5]))
            self.roi_hist = cv2.calcHist([self.roi], [0,1], None, [180,255], [0,180,0,255])
            cv2.normalize(self.roi_hist,self.roi_hist,0,255,cv2.NORM_MINMAX)
            self.track_window = (c,r,w,h)
            self.cnt=random.randint(10,100)
            self.has = True
        except:
            self.has = False
    def comp(self):
#         pts[1][1]-pts[0][1]
#         pts[1][0]-pts[0][0]
#         pts[2][1]-pts[1][1]
#         pts[2][0]-pts[1][0]
        if self.trackcount<=self.cnt:
            return False#remain
        else:
            return True#delete
    def track_csft(self):
        try:       
            dst = cv2.calcBackProject([hsv],[0,1],self.roi_hist,[0,180,0,255],1)
            self.ret, self.track_window = cv2.CamShift(dst, self.track_window, self.term_crit)
            self.trackcount += 1
        except:
            self.trackcount += 1
#         print(self.ret)
    def preTrackPred_csft(self):
        loadModel = None
    def plot(self):
        pts = cv2.boxPoints(self.ret)
        pts = np.int0(pts)
        return [pts]
#         img2 = cv2.polylines(frame,[pts],True, 255,2)
#         cv2.imshow('img2',img2)        
        
    
def sift_knn(img1,img2,part):
    
    global centroidsAll
    infolist = []
    sift = cv2.xfeatures2d.SURF_create()
    kp1 = sift.detect(img1,None)
    kp1, des1 = sift.compute(img1, kp1)
    kp2 = sift.detect(img2,None)
    kp2, des2 = sift.compute(img2, kp2)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    matchesMask = [[0,0] for i in range(len(matches))]
          
    for i, (m1,m2) in enumerate(matches):
        if m1.distance < 0.9 * m2.distance:
            matchesMask[i] = [1,0]
            pt1 = kp1[m1.queryIdx].pt
            pt2 = kp2[m1.trainIdx].pt
#             col = list(img2[int(pt2[1])-1 : int(pt2[1])+1,int(pt2[0])-1 : int(pt2[0])+1,:].flatten())
            info = [10*pt2[0], 10*pt2[1], 50*(pt2[1]-pt1[1]), 50*(pt2[0]-pt1[0])]#+col
#             print("yeah:",img2[int(pt2[0])-1 : int(pt2[0])+1,int(pt2[1])-1 : int(pt2[1])+1,:].flatten())
            infolist.append(info)
            cv2.circle(img1, (int(pt1[0]),int(pt1[1])), 1, (255,0,255), -1)
            cv2.circle(img2, (int(pt2[0]),int(pt2[1])), 1, (0,255,255), -1)



    draw_params = dict(matchColor = (255, 0,0),
                       singlePointColor = (0,0,255),
                       matchesMask = matchesMask,
                       flags = 2)
    res = cv2.drawMatchesKnn(img2,kp1,img2,kp2,matches,None,**draw_params)    
    
    infolist = np.array(infolist).reshape(-1,4)
    nc = min(infolist.shape[0],10)
    
    kmeans = KMeans(n_clusters = nc, n_init=10, max_iter=100)
    kmeans.fit(infolist)
    centroid = kmeans.cluster_centers_
    centroidsAll += list(centroid)
    for cnt in centroid:
        cv2.circle(img2,(int(cnt[0]/10),int(cnt[1]/10)),5,(0,0,255),-1)
    
        
def obstacleNcounters():
    PORT_NUMBER = 4000
    SIZE = 1024*1024
#    hostName = gethostbyname( '0.0.0.0' )
#    mySocket = socket( AF_INET, SOCK_DGRAM )
#    mySocket.bind( (hostName, PORT_NUMBER) )
    print ("Test server listening on port {0}\n".format(PORT_NUMBER))
    
    print("start")
    global centroidsAll, grayG
    vrec = cv2.VideoCapture('v4.mp4',0)
    k = 300*8
    prev1 = []
    shape = [260,680]
    cfstObj = []
    global loaded_model, hsv, scr
    
    with open("pathInertia.json", "r") as json_file:
        loaded_model_json = json_file.read()
        loaded_model = keras.models.model_from_json(loaded_model_json)
        loaded_model.load_weights("pathInertia.h5")
        print("Model loaded from disk")
###
    while k:
        ret, scr = vrec.read()
        k-=1
    print("running")
##    
    while True:
        
        start_time = time.time()
        centroidsAll = []
        ret, scr = vrec.read()
        scr1 = scr
        if not ret:
            break
    
        scr = cv2.resize(scr, (680,480)) 
        scr = np.array(scr[220:480,:])
        scr = cv2.resize(scr, (shape[1],shape[0]))#1360,520
        hsv = cv2.cvtColor(scr, cv2.COLOR_BGR2HSV)
        grayG = cv2.cvtColor(scr, cv2.COLOR_BGR2GRAY)
        blank = cv2.imread('blank.png', 0)
        blank = cv2.resize(blank,(shape[1],shape[0]))
        cv2.imshow('hsv0',grayG)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(gray,50,100)
        canny = blank - canny
        
        scr1 = canny[0:shape[0]//2, 0:shape[1]//3]
        scr2 = canny[0:shape[0]//2,shape[1]//3:2*shape[1]//3]
        scr3 = canny[0:shape[0]//2,2*shape[1]//3:shape[1]]
        scr4 = canny[shape[0]//2:shape[0], 0:shape[1]//3]
        scr5 = canny[shape[0]//2:shape[0],shape[1]//3:2*shape[1]//3]
        scr6 = canny[shape[0]//2:shape[0],2*shape[1]//3:shape[1]]
        if prev1 == [] :
            prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1 = threading.Thread(target = sift_knn, args=(prev1,scr1,0))#,0,0))
        th2 = threading.Thread(target = sift_knn, args=(prev2,scr2,1))#,0,shape[1]//3))
        th3 = threading.Thread(target = sift_knn, args=(prev3,scr3,2))#,0,2*shape[1]//3))
        th4 = threading.Thread(target = sift_knn, args=(prev4,scr4,3))#,shape[0]//2,0))
        th5 = threading.Thread(target = sift_knn, args=(prev5,scr5,4))#,shape[0]//2,shape[1]//3))
        th6 = threading.Thread(target = sift_knn, args=(prev6,scr6,5))#,shape[0]//2,2*shape[1]//3))
        th1.start()
        th2.start()
        th3.start()
        th4.start()
        th5.start()
        th6.start()
        
        frame = cv2.GaussianBlur(scr, (3, 3), 0)
        scr = cv2.addWeighted(scr,10,frame,-9,0)
        cv2.imshow('scred',scr)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        Cy = cv2.Canny(gray,50,100)
        Cy = blank - Cy
        im2, contours, hierarchy = cv2.findContours(Cy, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        blank = cv2.drawContours(blank, contours, -1, 0, 1)
        dist_transform = cv2.distanceTransform(Cy,cv2.DIST_L2,5)
        ret, blank = cv2.threshold(dist_transform,0.02*dist_transform.max(),255,0)
        blank = cv2.morphologyEx(blank, cv2.MORPH_OPEN, kernel,3)
        cv2.imshow("blk1",blank)
        blank = np.uint8(blank)
        im3, contours1, hierarchy = cv2.findContours(blank, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        scr = cv2.drawContours(scr, contours1, -1, 0, 1)
        cv2.imshow("blk2",scr)
        print("no",len(cfstObj))
        for obj in cfstObj:
            obj.track_csft()
            obj.preTrackPred_csft()
            pts = obj.plot()
            cv2.polylines(scr,pts,True, (0,0,255),1)  
            if obj.comp():
                cfstObj.remove(obj)
        
        prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1.join()
        th2.join()
        th3.join()
        th4.join()
        th5.join()
        th6.join()
        cv2.imshow('print',canny)
        for cnt in centroidsAll:
            point = Point(cnt[0]/10,cnt[1]/10)
            has=1
            for obj in cfstObj:
                pts = cv2.boxPoints(obj.ret)
                polygon = Polygon(np.int0(pts))
                if polygon.contains(point):
                    has=0
                    break
            if has:    
                tempobj = camshift(int(cnt[0]/10),int(cnt[1]/10))
                retCt = tempobj.has
#                 print(int(cnt[0]/10),int(cnt[1]/10))
                if retCt:
                    tempobj.track_csft()
                    tempobj.preTrackPred_csft()
                    cfstObj.append(tempobj)
            
                    
                
#         kmeanC = KMeans(n_clusters = 10, n_init=10, max_iter=100)
#         kmeanC.fit(centroidsAll)
#         centroid = kmeanC.cluster_centers_
#         for cnt in centroid:
#              cv2.circle(canny,(int(cnt[0]/2),int(cnt[1]/2)),10,(0,0,255),-1)
        cv2.imshow('print2',canny) 
        cv2.imshow('print3',scr)   
#         for  

        print("FPS: ", 1.0 / (time.time() - start_time))
        q=cv2.waitKey(1)
        if q == 27:
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
obstacleNcounters()

Test server listening on port 4000

start
Model loaded from disk
running
no 0
FPS:  1.1782864899608503
no 41


/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:185: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


FPS:  1.7498555451673234
no 77
FPS:  1.615469172349671
no 101
FPS:  1.8039699084876353
no 116
FPS:  1.9108873045457506
no 121
FPS:  1.9628122243358908
no 134
FPS:  1.6829380347324567
no 135
FPS:  2.095087291474252
no 135
FPS:  2.185143352519245
no 135
FPS:  1.6977125042804027
no 135
FPS:  1.9583765422731676
no 135
FPS:  1.9163658759907816
no 137
FPS:  2.059129150668334
no 139
FPS:  1.9106139805444902
no 138
FPS:  1.5251387852062805
no 140
FPS:  1.563638532657322
no 139
FPS:  2.1302345424441578
no 140
FPS:  2.0352172851858663
no 136
FPS:  1.6971787358998502
no 135
FPS:  2.31124620261779
no 134
FPS:  2.2963525461440883
no 134
FPS:  2.2881670404535415
no 132
FPS:  2.233574247914097
no 130
FPS:  2.3150873807628747
no 130
FPS:  2.1584853615483977
no 128
FPS:  2.1678196694960676
no 127
FPS:  2.1432500946610165
no 125
FPS:  2.045052248906849
no 123
FPS:  2.1866482321703855
no 120
FPS:  2.1025576757844417
no 118
FPS:  2.2033281817503685
no 118
FPS:  2.1028591597595874
no 118
FPS:  2.0932585391

Exception in thread Thread-978:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-746acf543c5c>", line 96, in sift_knn
    matches = flann.knnMatch(des1,des2,k=2)
cv2.error: OpenCV(3.4.1) /Users/travis/build/skvark/opencv-python/opencv/modules/flann/src/miniflann.cpp:315: error: (-210) type=0
 in function buildIndex_




FPS:  1.663348274887046
no 129


...

FPS:  1.8944702101020248
no 124


Exception in thread Thread-996:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-746acf543c5c>", line 123, in sift_knn
    kmeans.fit(infolist)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 884, in fit
    X = self._check_fit_data(X)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 858, in _check_fit_data
    X = check_array(X, accept_sparse='csr', dtype=[np.float64, np.float32])
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py", line 431, in check_array
    context))
ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.



FPS:  1.871219130914279
no 124


Exception in thread Thread-1002:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-746acf543c5c>", line 123, in sift_knn
    kmeans.fit(infolist)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 884, in fit
    X = self._check_fit_data(X)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 858, in _check_fit_data
    X = check_array(X, accept_sparse='csr', dtype=[np.float64, np.float32])
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py", line 431, in check_array
    context))
ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.



FPS:  1.8735193614236643
no 119
FPS:  1.8535714017402156
no 113
FPS:  1.8216652131463098
no 111
FPS:  1.9840915208231726
no 108
FPS:  2.3398154279392984
no 106
FPS:  2.11982136972787
no 106
FPS:  2.077970256561679
no 103
FPS:  2.1056398341715457
no 101
FPS:  2.1385206119784166
no 99
FPS:  2.2523949428430425
no 98
FPS:  2.137104092456427
no 97
FPS:  2.104325646127649
no 97
FPS:  2.1449094820730985
no 96
FPS:  2.118517246430249
no 93
FPS:  1.765801918665329
no 107
FPS:  1.7778312793396136
no 117
FPS:  1.6415567906564068
no 131
FPS:  1.7025592148499165
no 136
FPS:  1.7780973278753573
no 137
FPS:  1.8099351939994253
no 135
FPS:  1.6634122623543233
no 138
FPS:  1.625767910349701
no 144
FPS:  1.5782103849553344
no 153
FPS:  1.6508815674759756
no 160
FPS:  1.5597705064833258
no 164
FPS:  1.5903997476172276
no 168
FPS:  1.6187174080168947
no 167
FPS:  1.7394757731065185


In [59]:
import numpy as np
import time
import cv2
from pathInertiaModel import pathInertiaModel
from socket import socket, gethostbyname, AF_INET, SOCK_DGRAM
from sklearn.neighbors import KNeighborsClassifier
import copy
from random import randint
import skimage.measure
import keras
import math
import threading
import random
from skimage import io
from sklearn.cluster import KMeans
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon


kernel = np.ones((5,5),np.uint8)
blnk = cv2.imread('blank.png', 0)
blnk = cv2.resize(blnk,(260,680))
centroidsAll = [[0,0,0,0]]
scr = None
grayG = None
hsv=None

class camshift:
#     r,h,c,w = 0,10,360,10
    ret = [] 
    hsv_roi = None
    track_window = None
    mask = None
    roi_hist = None
    term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1 )
    coord = []
    trackcount = 0
    cnt = 10
    has = True
    global hsv, loadModel, scr, grayG
    def  __init__(self,cx, cy):
        try:
            x, y, h, w = (cx-20), (cy-20), 40, 40 
            self.roi = hsv[y:y+w, x:x+h,:]
#             mean = np.mean(self.roi)
# #             self.hsv_roi =  cv2.cvtColor(self.roi, cv2.COLOR_BGR2GRAY)
#             mask = cv2.inRange(self.roi,np.array([mean-5]), np.array([mean+5]))
            self.roi_hist = cv2.calcHist([self.roi], [0,1], None, [180,255], [0,180,0,255])
            cv2.normalize(self.roi_hist,self.roi_hist,0,255,cv2.NORM_MINMAX)
            self.track_window = (x,y,w,h)
            self.cnt=random.randint(10,50)
            self.has = True
        except:
            self.has = False
    def comp(self):
#         pts[1][1]-pts[0][1]
#         pts[1][0]-pts[0][0]
#         pts[2][1]-pts[1][1]
#         pts[2][0]-pts[1][0]
        if self.trackcount<=self.cnt:
            return False#remain
        else:
            return True#delete
    def track_csft(self):
        try:       
            dst = cv2.calcBackProject([hsv],[0,1],self.roi_hist,[0,180,0,255],1)
            self.ret, self.track_window = cv2.CamShift(dst, self.track_window, self.term_crit)
            self.trackcount += 1
        except:
            self.trackcount += 1
#         print(self.ret)
    def preTrackPred_csft(self):
        loadModel = None
    def plot(self):
        pts = cv2.boxPoints(self.ret)
        pts = np.int0(pts)
        return [pts]
#         img2 = cv2.polylines(frame,[pts],True, 255,2)
#         cv2.imshow('img2',img2)        
        
    
def sift_knn(img1,img2,part1,part2):
    
    global centroidsAll
    infolist = []
    sift = cv2.xfeatures2d.SURF_create()
    kp1 = sift.detect(img1,None)
    kp1, des1 = sift.compute(img1, kp1)
    kp2 = sift.detect(img2,None)
    kp2, des2 = sift.compute(img2, kp2)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    matchesMask = [[0,0] for i in range(len(matches))]
          
    for i, (m1,m2) in enumerate(matches):
        if m1.distance < 0.9 * m2.distance:
            matchesMask[i] = [1,0]
            pt1 = kp1[m1.queryIdx].pt
            pt2 = kp2[m1.trainIdx].pt
#             col = list(img2[int(pt2[1])-1 : int(pt2[1])+1,int(pt2[0])-1 : int(pt2[0])+1,:].flatten())
            info = [10*pt2[0], 10*pt2[1], 50*(pt2[1]-pt1[1]), 50*(pt2[0]-pt1[0])]#+col
#             print("yeah:",img2[int(pt2[0])-1 : int(pt2[0])+1,int(pt2[1])-1 : int(pt2[1])+1,:].flatten())
            infolist.append(info)
            cv2.circle(img1, (int(pt1[0]),int(pt1[1])), 1, (255,0,255), -1)
            cv2.circle(img2, (int(pt2[0]),int(pt2[1])), 1, (0,255,255), -1)



    draw_params = dict(matchColor = (255, 0,0),
                       singlePointColor = (0,0,255),
                       matchesMask = matchesMask,
                       flags = 2)
    res = cv2.drawMatchesKnn(img2,kp1,img2,kp2,matches,None,**draw_params)    
    
    infolist = np.array(infolist).reshape(-1,4)
    nc = min(infolist.shape[0],10)
    
    kmeans = KMeans(n_clusters = nc, n_init=10, max_iter=100)
    kmeans.fit(infolist)
    centroid = kmeans.cluster_centers_
        
    for inx in range(len(centroid)):
        cv2.circle(img2,(int(centroid[inx][0]/10),int(centroid[inx][1]/10)),5,(0,0,255),-1)
#         centroid[inx][0]+=part1
#         centroid[inx][1]+=part2
    centroidsAll += list(np.array(centroid)+[10*part2,10*part1,0,0])
    
    
        
def obstacleNcounters():
    PORT_NUMBER = 4000
    SIZE = 1024*1024
#    hostName = gethostbyname( '0.0.0.0' )
#    mySocket = socket( AF_INET, SOCK_DGRAM )
#    mySocket.bind( (hostName, PORT_NUMBER) )
    print ("Test server listening on port {0}\n".format(PORT_NUMBER))
    
    print("start")
    global centroidsAll, grayG
    vrec = cv2.VideoCapture('v4.mp4',0)
    k = 300*8
    prev1 = []
    shape = [260,680]
    cfstObj = []
    global loaded_model, hsv, scr
    
    with open("pathInertia.json", "r") as json_file:
        loaded_model_json = json_file.read()
        loaded_model = keras.models.model_from_json(loaded_model_json)
        loaded_model.load_weights("pathInertia.h5")
        print("Model loaded from disk")
###
    while k:
        ret, scr = vrec.read()
        k-=1
    print("running")
##    
    while True:
        
        start_time = time.time()
        centroidsAll = []
        ret, scr = vrec.read()
        scr1 = scr
        if not ret:
            break
    
        scr = cv2.resize(scr, (680,480)) 
        scr = np.array(scr[220:480,:])
        scr = cv2.resize(scr, (shape[1],shape[0]))#1360,520
        hsv = cv2.cvtColor(scr, cv2.COLOR_BGR2HSV)
        grayG = cv2.cvtColor(scr, cv2.COLOR_BGR2GRAY)
        blank = cv2.imread('blank.png', 0)
        blank = cv2.resize(blank,(shape[1],shape[0]))
        cv2.imshow('hsv0',grayG)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(gray,50,100)
        canny = blank - canny
        
        scr1 = canny[0:shape[0]//2, 0:shape[1]//3]
        scr2 = canny[0:shape[0]//2,shape[1]//3:2*shape[1]//3]
        scr3 = canny[0:shape[0]//2,2*shape[1]//3:shape[1]]
        scr4 = canny[shape[0]//2:shape[0], 0:shape[1]//3]
        scr5 = canny[shape[0]//2:shape[0],shape[1]//3:2*shape[1]//3]
        scr6 = canny[shape[0]//2:shape[0],2*shape[1]//3:shape[1]]
        if prev1 == [] :
            prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1 = threading.Thread(target = sift_knn, args=(prev1,scr1,0,0))#,0,0))
        th2 = threading.Thread(target = sift_knn, args=(prev2,scr2,0,shape[1]//3))#,0,shape[1]//3))
        th3 = threading.Thread(target = sift_knn, args=(prev3,scr3,0,2*shape[1]//3))#,0,2*shape[1]//3))
        th4 = threading.Thread(target = sift_knn, args=(prev4,scr4,shape[0]//2,0))#,shape[0]//2,0))
        th5 = threading.Thread(target = sift_knn, args=(prev5,scr5,shape[0]//2,shape[1]//3))#,shape[0]//2,shape[1]//3))
        th6 = threading.Thread(target = sift_knn, args=(prev6,scr6,shape[0]//2,2*shape[1]//3))#,shape[0]//2,2*shape[1]//3))
        th1.start()
        th2.start()
        th3.start()
        th4.start()
        th5.start()
        th6.start()
        
        frame = cv2.GaussianBlur(scr, (3, 3), 0)
        scr = cv2.addWeighted(scr,10,frame,-9,0)
        cv2.imshow('scred',scr)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        Cy = cv2.Canny(gray,50,100)
        Cy = blank - Cy
        im2, contours, hierarchy = cv2.findContours(Cy, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        blank = cv2.drawContours(blank, contours, -1, 0, 1)
        dist_transform = cv2.distanceTransform(Cy,cv2.DIST_L2,5)
        ret, blank = cv2.threshold(dist_transform,0.02*dist_transform.max(),255,0)
        blank = cv2.morphologyEx(blank, cv2.MORPH_OPEN, kernel,3)
        cv2.imshow("blk1",blank)
        blank = np.uint8(blank)
        im3, contours1, hierarchy = cv2.findContours(blank, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        scr = cv2.drawContours(scr, contours1, -1, 0, 1)
        cv2.imshow("blk2",scr)
#         print("no",len(cfstObj))
        for obj in cfstObj:
            obj.track_csft()
            obj.preTrackPred_csft()
            pts = obj.plot()
            cv2.polylines(scr,pts,True, (0,0,255),2) 
            if obj.comp():
                cfstObj.remove(obj)
        
        prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1.join()
        th2.join()
        th3.join()
        th4.join()
        th5.join()
        th6.join()
        cv2.imshow('print',canny)
        for cnt in centroidsAll:
            point = Point(cnt[0]/10,cnt[1]/10)
            has=1
            for obj in cfstObj:
                pts = cv2.boxPoints(obj.ret)
                polygon = Polygon(pts)
                if polygon.contains(point):
                    has=0
                    break
            if has:    
                tempobj = camshift(int(cnt[0]/10),int(cnt[1]/10))
                retCt = tempobj.has
                if retCt:  
                    tempobj.track_csft()
#                     x,y,w,h = tempobj.track_window        
#                     cv2.rectangle(scr, (x,y), (x+w,y+h), [225,0,255],2)
#                     font = cv2.FONT_HERSHEY_SIMPLEX
#                     cv2.putText(scr,str(x)+str(y),(x,y), font, 1,(255,255,0),1,cv2.LINE_AA)

#                     tempobj.preTrackPred_csft()
                    cfstObj.append(tempobj)
            
                    
                
#         kmeanC = KMeans(n_clusters = 10, n_init=10, max_iter=100)
#         kmeanC.fit(centroidsAll)
#         centroid = kmeanC.cluster_centers_
#         for cnt in centroid:
#              cv2.circle(canny,(int(cnt[0]/2),int(cnt[1]/2)),10,(0,0,255),-1)
        cv2.imshow('print2',canny) 
        cv2.imshow('print3',scr)   
 

        print("FPS: ", 1.0 / (time.time() - start_time))
        q=cv2.waitKey(1)
        if q == 27:
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
obstacleNcounters()

Test server listening on port 4000

start
Model loaded from disk
running
FPS:  1.0419282373635612


/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:189: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


FPS:  2.150509363061603
FPS:  2.126194757287164
FPS:  2.172652770086905
FPS:  2.3699953439803996
FPS:  2.3611298600481763
FPS:  2.390069337758307
FPS:  2.2653594714760388
FPS:  2.3854612296804034
FPS:  2.3446103863072154
FPS:  2.41696617241735
FPS:  2.4498151382228737
FPS:  2.401001088787096
FPS:  2.371478730892859
FPS:  2.3682530289872585
FPS:  2.4086881066866974
FPS:  2.5154514530267194
FPS:  2.503016052992779
FPS:  2.439446610293207
FPS:  2.5714683344236042
FPS:  2.4707082654929393
FPS:  2.375664177705379
FPS:  2.480036092139021
FPS:  2.430777415692504
FPS:  2.3426225305526995
FPS:  2.3015390220869176
FPS:  2.1593087829722477
FPS:  2.134180023406096
FPS:  2.109656991819007
FPS:  2.135461104483889
FPS:  2.138777966166516
FPS:  2.1637243901747
FPS:  2.0934560034259513
FPS:  2.1351599828547982
FPS:  2.3260737069015716
FPS:  2.215846007222902
FPS:  2.305208602410566
FPS:  2.2938345496228627
FPS:  2.4217416684325124
FPS:  2.3265936899187136
FPS:  2.4120846227235946
FPS:  2.36507209732970

Exception in thread Thread-18276:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-59-5242b9d967bf>", line 96, in sift_knn
    matches = flann.knnMatch(des1,des2,k=2)
cv2.error: OpenCV(3.4.1) /Users/travis/build/skvark/opencv-python/opencv/modules/flann/src/miniflann.cpp:315: error: (-210) type=0
 in function buildIndex_




FPS:  3.0931399806194975


...

FPS:  3.010649202242965


Exception in thread Thread-18294:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-59-5242b9d967bf>", line 123, in sift_knn
    kmeans.fit(infolist)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 884, in fit
    X = self._check_fit_data(X)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 858, in _check_fit_data
    X = check_array(X, accept_sparse='csr', dtype=[np.float64, np.float32])
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py", line 431, in check_array
    context))
ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.



FPS:  3.022479593976521


Exception in thread Thread-18300:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-59-5242b9d967bf>", line 123, in sift_knn
    kmeans.fit(infolist)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 884, in fit
    X = self._check_fit_data(X)
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py", line 858, in _check_fit_data
    X = check_array(X, accept_sparse='csr', dtype=[np.float64, np.float32])
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py", line 431, in check_array
    context))
ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.



FPS:  2.9401484127573845
FPS:  2.610298917930906
FPS:  2.7647162350378984
FPS:  2.8759331191738977
FPS:  2.740085711299258
FPS:  2.5321515872493614
FPS:  2.131446981939405
FPS:  2.5905941196416915
FPS:  2.8306879949464343
FPS:  2.4519361233338537
FPS:  2.6843047393040385
FPS:  2.7058629232039637
FPS:  2.511452507108979
FPS:  2.434019014612921
FPS:  2.6109391291897897
FPS:  2.6798952143632992
FPS:  2.5954337537870535
FPS:  2.542871623997536
FPS:  2.648081353466721
FPS:  2.621666118702738
FPS:  2.5145466230539757
FPS:  2.5673413049399683
FPS:  2.6268217009598414
FPS:  2.675565007881309
FPS:  2.6633359389039204
FPS:  2.5930637789341513
FPS:  2.686374713784766
FPS:  2.53197121464839
FPS:  2.46631640534271
FPS:  2.649397769967583
FPS:  2.3399485739565975
